In [1]:
import mcnpy
import sys
import os
from mcnpy.utils import configure_ace_debug_logging

# Configure debug logging
configure_ace_debug_logging(enable=True)  # Set to False to disable debug output

<RootLogger root (DEBUG)>

In [2]:
# Define all ACE files to test
acefile = 'files/10010_71.02c'
acefile_U235 = 'files/922350_71.02c'
acefile1 = 'files/FE56.293600c'
acefile2 = 'files/260560_80.02c'

# Print files to verify they exist
for file_path in [acefile, acefile_U235, acefile1, acefile2]:
    print(f"File {file_path} exists: {os.path.exists(file_path)}")

File files/10010_71.02c exists: True
File files/922350_71.02c exists: True
File files/FE56.293600c exists: True
File files/260560_80.02c exists: True


In [3]:
loc = 4432756+138
print((loc)/4+13-0.25)

1108236.25


In [4]:
#ace_U235.header.jxs_array[24]

In [5]:
ace1 = mcnpy.read_ace(acefile1, True)

DEBUG - mcnpy.ace.parsers.parse_ace - Reading ACE file: files/FE56.293600c
DEBUG - mcnpy.ace.parsers.parse_ace - ACE format version: legacy
DEBUG - mcnpy.ace.parsers.parse_angular_distribution - First few values from angular distribution for MT=51:
DEBUG - mcnpy.ace.parsers.parse_angular_distribution -   First 3 cosine grid values: [-1.0, 0.0, 1.0]
DEBUG - mcnpy.ace.parsers.parse_energy_distribution_locators - Reading energy distribution locator blocks
DEBUG - mcnpy.ace.parsers.parse_energy_distribution_locators - Secondary neutron reactions: 75
DEBUG - mcnpy.ace.parsers.parse_energy_distribution_locators - Photon production reactions: 42
DEBUG - mcnpy.ace.parsers.parse_energy_distribution_locators - Particle types: 532
DEBUG - mcnpy.ace.parsers.parse_energy_distribution_locators - Delayed neutron precursors: 5
DEBUG - mcnpy.ace.parsers.parse_energy_distribution_locators - LDLW block index: 3110994
DEBUG - mcnpy.ace.parsers.parse_energy_distribution_locators - LDLWP block index: 431394

In [41]:
ace1.header.jxs_array[13]

4307320

In [7]:
ace1.xss_data[ace1.header.jxs_array[31]+1-1]

XssEntry(Index=4405339, Value=13.0)

In [8]:
ace1.xss_data[ace1.header.jxs_array[32]+10*(1-1)+6]

XssEntry(Index=4405350, Value=4432756.0)

In [9]:
ace1.header.nxs_array[6]

532

In [10]:
ace1

                                   ACE Format Data                                   
                                   Material: 2631                                    
                         ZA: 26056, Temperature: 2.53e-08 K                          

ACE Data Components:
-------------------------------------------------------------------------------------
Component                                Access Information                        
-------------------------------------------------------------------------------------
Header Information                       Available: ace.header                     
Energy Grid & Cross Sections             Available: ace.esz_block                  
Nubar (ν) Data                           Available: ace.nubar                      
Delayed Neutron Data                     Not available                             
Reaction MT Numbers                      Available: ace.reaction_mt_data           
Reaction Q-values                        Ava

In [11]:
ace1.angular_distributions 

                              Angular Distribution Container                              

This container holds angular distributions for different reaction types and particles.
Angular distributions describe the probability of a particle scattering at a specific angle,
represented by the cosine of the scattering angle (μ) ranging from -1 to +1.

Note: Some distributions (Kalbach-Mann/Law=44) require additional data from the energy
distribution section. For these distributions, the ACE object must be provided when
calling methods to avoid Law44DataError exceptions.

Available Angular Distribution Data:
------------------------------------------------------------------------------------------
Distribution Type                        Status                                         
------------------------------------------------------------------------------------------
Elastic Scattering (MT=2)                Available                                      
Neutron Reactions            

In [12]:
ace1.particle_production_xs.get_available_mts()

[5,
 111,
 600,
 601,
 602,
 603,
 604,
 605,
 606,
 607,
 608,
 609,
 649,
 800,
 801,
 802,
 803,
 804,
 805,
 806,
 807,
 808,
 809,
 810,
 811,
 812,
 813,
 814,
 815,
 816,
 817,
 818]

In [13]:
ace1.angular_distributions.incident_neutron[16]

                      Kalbach-Mann Angular Distribution Details                      

This object represents an angular distribution using the Kalbach-Mann formalism.
The Kalbach-Mann model correlates energy and angle distributions, with parameters
R (precompound fraction) and A (angular slope) that vary with outgoing energy.

IMPORTANT: This distribution REQUIRES Law=44 data from the energy distribution
section (DLW/DLWH blocks). The ACE object must be provided to all methods that
calculate or sample angular distributions. Without this data, methods will raise
a Law44DataError exception.

Data Information:
-------------------------------------------------------------------------------------
Property                            Value                                          
-------------------------------------------------------------------------------------
MT Number                           16                                             
Distribution Type                   Kalbach-

In [14]:
#ace = mcnpy.read_ace(acefile)
ace_U235 = mcnpy.read_ace(acefile_U235, True)

DEBUG - mcnpy.ace.parsers.parse_ace - Reading ACE file: files/922350_71.02c
DEBUG - mcnpy.ace.parsers.parse_ace - ACE format version: legacy
DEBUG - mcnpy.ace.parsers.parse_energy_distribution_locators - Reading energy distribution locator blocks
DEBUG - mcnpy.ace.parsers.parse_energy_distribution_locators - Secondary neutron reactions: 48
DEBUG - mcnpy.ace.parsers.parse_energy_distribution_locators - Photon production reactions: 45
DEBUG - mcnpy.ace.parsers.parse_energy_distribution_locators - Particle types: 33
DEBUG - mcnpy.ace.parsers.parse_energy_distribution_locators - Delayed neutron precursors: 0
DEBUG - mcnpy.ace.parsers.parse_energy_distribution_locators - LDLW block index: 684913
DEBUG - mcnpy.ace.parsers.parse_energy_distribution_locators - LDLWP block index: 835400
DEBUG - mcnpy.ace.parsers.parse_energy_distribution_locators - DNEDL block index: 747714
DEBUG - mcnpy.ace.parsers.parse_energy_distribution_locators - Particle pointer index: -1
DEBUG - mcnpy.ace.parsers.parse_

In [15]:
ace_U235.unresolved_resonance

Unresolved Resonance Probability Tables
Number of energy points: 14
Table length: 20
Interpolation: linear-linear
Inelastic cross section: special MT=4
Other absorption cross section: zero in the unresolved range
Table values represent factors to multiply smooth cross sections
Energy range: 2.250001e-03 to 2.499999e-02 MeV
Total cross section range: 6.280051e-01 to 1.891055e+00 (factors)

In [16]:
ace_U235.header.nxs_array[7]

0

In [17]:
# ace = mcnpy.read_ace(acefile)
# ace_U235 = mcnpy.read_ace(acefile_U235)

In [18]:
# ace_U235.angular_distributions

In [19]:
# ace_U235.angular_distributions.incident_neutron[16]

In [20]:
# ace_U235.angular_distributions.to_dataframe(energy=2, mt=16)

In [21]:
# ace_U235.reaction_mt_data

In [22]:
# ace_U235.energy_distributions

In [23]:
# Print all neutron reactions with energy distributions
# for mt in ace_U235.energy_distributions.get_neutron_reaction_mt_numbers():
#     distributions = ace_U235.energy_distributions.get_neutron_distribution(mt)
#     for dist in distributions:
#         print(f"MT={mt}, {dist.law_name} (Law {dist.law})")

In [24]:
# ace.reaction_mt_data

In [25]:
# ace.particle_release

In [26]:
# ace_U235.reaction_mt_data.incident_neutron

In [27]:
# print(f"Elastic locator: {ace.angular_locators.elastic_scattering}")

In [28]:
# ace.angular_distributions

In [29]:
# ace_U235.header.nxs_array[3]

In [30]:
# ace_U235

In [31]:
# ace_U235.angular_distributions

In [32]:
# ace_U235.angular_distributions.incident_neutron[16]

In [33]:
# ace_U235.angular_distributions.incident_neutron[16].plot(16,ace_U235)

In [34]:
# mu = ace_U235.angular_distributions.elastic.sample_mu(energy=1.0, random_value=0.5)
# mu

In [35]:
# ace_U235.get_cross_section(1)

In [36]:
# ace_U235.plot_cross_section(reactions=[1,2,4])

In [37]:
# write_ace(ace, filepath=acefile+'_recon', overwrite=True)

In [38]:
# write_ace(ace_U235, acefile_U235+'_recon', overwrite=True)

In [39]:
# Test particle production data parsing
print("Particle types in U-235:")
if hasattr(ace_U235, 'particle_types') and ace_U235.particle_types is not None:
    print(f"Has particle types data: {ace_U235.particle_types.has_data}")
    if hasattr(ace_U235.particle_types, 'particle_ids'):
        print(f"Particle IDs: {ace_U235.particle_types.particle_ids}")
    else:
        print("No particle_ids attribute")
else:
    print("No particle_types attribute")

print("\nSecondary particles in U-235:")
if hasattr(ace_U235, 'secondary_particles') and ace_U235.secondary_particles is not None:
    print(f"Has secondary particles data: {ace_U235.secondary_particles.has_data}")
    if hasattr(ace_U235.secondary_particles, 'particle_ids'):
        print(f"Secondary particle IDs: {ace_U235.secondary_particles.particle_ids}")
    else:
        print("No particle_ids attribute")
else:
    print("No secondary_particles attribute")

Particle types in U-235:
No particle_types attribute

Secondary particles in U-235:
Has secondary particles data: False
Secondary particle IDs: []
